In [1]:
#standard libraries
import csv
import os
import time

#custom libraries
import tensorflow as tf
import pandas as pd

In [28]:
def read_file_format(filename_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    _, value = reader.read(filename_queue)

    record_defaults = [tf.constant([], dtype=tf.float32), tf.constant([], dtype=tf.int32)]
    _, col2 = tf.decode_csv(value, record_defaults=record_defaults)
#     col2 = tf.string_to_number(col2, out_type=tf.int32)
    
    example = tf.stack([col2])
    return example

In [22]:
def input_pipeline(filenames, batch_size = 3, num_epochs = None, evaluation = False):   
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs, shuffle=False)

    example = read_file_format(filename_queue)
        
    return example

In [4]:
def _activation_summary(x):
    tensor_name = x.name
    tensor_name = tensor_name.replace(':', '_')
    tensor_name = tensor_name.replace('(', '_')
    tensor_name = tensor_name.replace(')', '_')
    tensor_name = tensor_name.replace(' ', '_')

    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))

In [5]:
#Need to factor so a block can replicate cells 
class LSTM_Block:
    
    def __init__(self, activation=tf.tanh, forget_bias=1.0, num_cells=1):
        self.block_num = block_num
        self.forget_bias = forget_bias
        self.activation = activation
        self.num_cells = num_cells
        self.state = tf.zeroes([batch_size, tf.shape(i_tensor)])
    
    def call(self, input_tensor):
        for c in range(num_cells):
            with tf.variable_scope('input{0}.{1}'.format(block_num, c)) as scope:
                i_tensor = tf.concat([input_tensor, self.cell_value])

                weights = tf.Variable(tf.random_normal(tf.shape(i_tensor)), name='weights', trainable=True)
                biases = tf.Variable(tf.random_normal(tf.shape(i_tensor)[1]), name='bias', trainable=True)

                lin_i = tf.matmul(i_tensor, weights) + biases
                _activation_summary(lin_i)

                gate_i = tf.sigmoid(lin_i)
                _activation_summary(gate_i)

            with tf.variable_scope('forget{0}.{1}'.format(block_num, c)) as scope:
                f_tensor = tf.concat([input_tensor, self.cell_value])

                weights = tf.Variable(tf.random_normal(tf.shape(f_tensor)), name='weights', trainable=True)
                biases = tf.Variable(tf.random_normal(tf.shape(f_tensor)[1]), name='bias', trainable=True)

                lin_f = tf.matmul(f_tensor, weights) + biases
                _activation_summary(lin_f)

                gate_f = tf.sigmoid(lin_f)
                _activation_summary(gate_f)

            with tf.variable_scope('cell{0}.{1}'.format(block_num, c)) as scope:
                weights = tf.Variable(tf.random_normal(tf.shape(input_tensor)), name='weights', trainable=True)
                biases = tf.Variable(tf.random_normal(tf.shape(input_tensor)[1]), name='bias', trainable=True)

                lin_state = tf.matmul(input_tensor, weights) + biases
                _activation_summary(lin_c)

                input_state = tf.matmul(gate_i, activation(lin_state))
                past_state = tf.matmul(gate_f, self.state)
                new_state = input_state + past_state
                _activation_summary(new_state)

                self.state = new_state

            with tf.variable_scope('output{0}.{1}'.format(block_num, c)) as scope:
                o_tensor = tf.concat([input_tensor, self.cell_value])

                weights = tf.Variable(tf.random_normal(tf.shape(o_tensor)), name='weights', trainable=True)
                biases = tf.Variable(tf.random_normal(tf.shape(o_tensor)[1]), name='bias', trainable=True)

                lin_o = tf.matmul(o_tensor, weights) + biases
                _activation_summary(lin_o)

                gate_o = tf.sigmoid(lin_o)
                _activation_summary(gate_o)

            with tf.variable_scope('hidden{0}.{1}'.format(block_num, c)) as scope:
                hidden = tf.matmul(gate_o, activation(self.cell_value))
                _activation_summary(hidden)

In [6]:
def make_prediction(X):
    block1 = lstm_block()

In [ ]:
def calculate_loss(logits, labels):
    NLLCriterion = -tf.reduce_mean(tf.reduce_sum(tf.multiply(labels, tf.log(logits + 1e-10)), axis=1))

    tf.add_to_collection('losses', NLLCriterion)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [20]:
pd.read_csv("{0}/{1}".format(data_path, train_file), nrows=10, header=None)

,0,1
0,NaN,Sequence
1,0.0,13
2,1.0,10
3,2.0,31
4,3.0,14
5,4.0,1
6,5.0,10
7,6.0,23
8,7.0,14
9,8.0,12


In [7]:
global data_path
data_path = "data"

global vocab_file
vocab_file = "vocab1.csv"

global train_file
train_file = "train1.csv"

global batch_size
batch_size = 16

global seq_length
seq_length = 16

global num_epochs
num_epochs = 1

In [30]:
with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, shape=[None, 1])
    y_ = tf.placeholder(tf.float32, shape = [None, 1])

    y_hat = make_prediction(x)

    #need appropriate loss function
    loss = calculate_loss(y_hat, y_)

    #need to figure out gradient clipping in TF
    #need trunc. backprop through time
    train_op = train(loss, global_step=global_step)
    step = 0

    accuracy = evaluate_accuracy(y_hat, y_)

    example_feed = input_pipeline(["{0}/{1}".format(data_path, train_file)],
                                  batch_size = batch_size, num_epochs = num_epochs)
    
    with tf.Session() as sess:
        # Start populating the filename queue.
        
        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)
        coord = tf.train.Coordinator()  
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
        
        step = 0
        while not coord.should_stop():
#             try:
            start_time = time.time()                

            example_batch = sess.run(example_feed)
        
            duration = time.time() - start_time
            step += 1

            except (tf.errors.OutOfRangeError, tf.errors.InvalidArgumentError) as e:
           
                print('Done training for %d epochs, %d steps.' % (num_epochs, step))
                # When done, ask the threads to stop.
                coord.request_stop()

        
        # Wait for threads to finish.
        coord.join(threads)
        sess.close()

[13]
[10]
[31]
[14]
[1]
[10]
[23]
[14]
[12]
[20]
[28]
[29]
[14]
[18]
[23]
[5]
[1]
[28]
[18]
[22]
[22]
[24]
[23]
[28]
[1]
[27]
[14]
[28]
[14]
[10]
[27]
[12]
[17]
[5]
[1]
[10]
[23]
[1]
[14]
[33]
[25]
[14]
[27]
[18]
[10]
[23]
[1]
[12]
[24]
[22]
[25]
[10]
[23]
[34]
[1]
[29]
[17]
[14]
[34]
[1]
[17]
[10]
[31]
[14]
[1]
[23]
[24]
[29]
[1]
[11]
[14]
[14]
[23]
[1]
[12]
[17]
[10]
[27]
[16]
[14]
[13]
[1]
[24]
[27]
[1]
[15]
[24]
[27]
[22]
[10]
[21]
[21]
[34]
[1]
[10]
[27]
[27]
[14]
[28]
[29]
[14]
[13]
[6]
[1]
[18]
[27]
[10]
[23]
[1]
[18]
[28]
[23]
[29]
[1]
[22]
[10]
[20]
[18]
[23]
[16]
[1]
[10]
[23]
[1]
[10]
[29]
[24]
[22]
[18]
[12]
[1]
[11]
[24]
[22]
[11]
[5]
[1]
[23]
[24]
[29]
[1]
[10]
[29]
[1]
[10]
[21]
[21]
[5]
[1]
[12]
[17]
[10]
[31]
[14]
[35]
[1]
[28]
[10]
[18]
[13]
[1]
[22]
[24]
[23]
[13]
[10]
[34]
[6]
[1]
[29]
[17]
[14]
[1]
[19]
[10]
[25]
[10]
[23]
[14]
[28]
[14]
[22]
[10]
[13]
[14]
[1]
[29]
[18]
[23]
[1]
[27]
[24]
[11]
[24]
[29]
[28]
[1]
[17]
[10]
[31]
[14]
[1]
[11]
[21]
[24]
[12]
[20]
[34

[32]
[17]
[10]
[29]
[1]
[29]
[17]
[14]
[34]
[1]
[22]
[18]
[28]
[28]
[14]
[13]
[1]
[29]
[17]
[14]
[1]
[15]
[18]
[27]
[28]
[29]
[1]
[29]
[18]
[22]
[14]
[1]
[10]
[27]
[24]
[30]
[23]
[13]
[6]
[1]
[32]
[17]
[14]
[23]
[1]
[18]
[1]
[10]
[28]
[20]
[14]
[13]
[1]
[27]
[18]
[16]
[16]
[18]
[24]
[1]
[10]
[11]
[24]
[30]
[29]
[1]
[16]
[24]
[31]
[10]
[23]
[5]
[1]
[17]
[14]
[1]
[13]
[14]
[28]
[12]
[27]
[18]
[11]
[14]
[13]
[1]
[17]
[18]
[22]
[5]
[1]
[18]
[23]
[1]
[14]
[15]
[15]
[14]
[12]
[29]
[5]
[1]
[10]
[28]
[1]
[10]
[1]
[12]
[17]
[18]
[25]
[1]
[24]
[15]
[15]
[1]
[29]
[17]
[14]
[1]
[20]
[27]
[14]
[23]
[28]
[1]
[11]
[21]
[24]
[12]
[20]
[6]
[1]
[29]
[17]
[14]
[1]
[12]
[17]
[10]
[18]
[27]
[22]
[10]
[23]
[1]
[24]
[15]
[1]
[29]
[17]
[14]
[1]
[32]
[10]
[34]
[28]
[1]
[10]
[23]
[13]
[1]
[22]
[14]
[10]
[23]
[28]
[1]
[12]
[24]
[22]
[22]
[18]
[29]
[29]
[14]
[14]
[1]
[25]
[30]
[29]
[1]
[15]
[24]
[27]
[29]
[17]
[1]
[21]
[14]
[16]
[18]
[28]
[21]
[10]
[29]
[18]
[24]
[23]
[1]
[29]
[17]
[10]
[29]
[1]
[25]
[10]
[28]
[2

[1]
[19]
[24]
[30]
[27]
[23]
[10]
[21]
[1]
[27]
[14]
[25]
[24]
[27]
[29]
[14]
[13]
[6]
[1]
[13]
[14]
[22]
[24]
[12]
[27]
[10]
[29]
[28]
[1]
[28]
[10]
[34]
[1]
[13]
[14]
[15]
[14]
[23]
[28]
[14]
[1]
[13]
[24]
[21]
[21]
[10]
[27]
[28]
[1]
[28]
[17]
[24]
[30]
[21]
[13]
[1]
[11]
[14]
[1]
[30]
[28]
[14]
[13]
[1]
[29]
[24]
[1]
[11]
[27]
[18]
[23]
[16]
[1]
[29]
[27]
[24]
[24]
[25]
[28]
[1]
[17]
[24]
[22]
[14]
[1]
[10]
[23]
[13]
[1]
[27]
[14]
[25]
[10]
[18]
[27]
[1]
[29]
[17]
[14]
[1]
[27]
[14]
[10]
[13]
[18]
[23]
[14]
[28]
[28]
[1]
[24]
[15]
[1]
[29]
[17]
[14]
[1]
[10]
[27]
[22]
[14]
[13]
[1]
[15]
[24]
[27]
[12]
[14]
[28]
[5]
[1]
[10]
[23]
[13]
[1]
[22]
[24]
[27]
[14]
[1]
[22]
[24]
[23]
[14]
[34]
[1]
[28]
[17]
[24]
[30]
[21]
[13]
[1]
[11]
[14]
[1]
[13]
[18]
[27]
[14]
[12]
[29]
[14]
[13]
[1]
[29]
[24]
[1]
[13]
[24]
[22]
[14]
[28]
[29]
[18]
[12]
[1]
[25]
[27]
[24]
[19]
[14]
[12]
[29]
[28]
[6]
[1]
[19]
[24]
[23]
[10]
[29]
[17]
[10]
[23]
[1]
[21]
[6]
[1]
[20]
[14]
[22]
[25]
[23]
[14]
[27]
[5]
[1]

[27]
[29]
[18]
[23]
[16]
[1]
[29]
[17]
[14]
[1]
[18]
[23]
[23]
[24]
[31]
[10]
[29]
[18]
[24]
[23]
[1]
[18]
[23]
[1]
[10]
[30]
[16]
[22]
[14]
[23]
[29]
[14]
[13]
[1]
[13]
[14]
[28]
[18]
[16]
[23]
[1]
[12]
[10]
[29]
[14]
[16]
[24]
[27]
[34]
[1]
[10]
[28]
[1]
[25]
[10]
[27]
[29]
[1]
[24]
[15]
[1]
[18]
[29]
[28]
[1]
[28]
[25]
[24]
[23]
[28]
[24]
[27]
[28]
[17]
[18]
[25]
[6]
[1]
[11]
[30]
[29]
[1]
[28]
[24]
[22]
[14]
[1]
[24]
[15]
[1]
[29]
[17]
[14]
[1]
[16]
[27]
[14]
[10]
[29]
[14]
[28]
[29]
[1]
[15]
[24]
[27]
[29]
[30]
[23]
[14]
[28]
[1]
[18]
[23]
[1]
[17]
[18]
[28]
[29]
[24]
[27]
[34]
[1]
[32]
[14]
[27]
[14]
[1]
[22]
[10]
[13]
[14]
[1]
[32]
[17]
[14]
[23]
[1]
[27]
[18]
[28]
[20]
[29]
[10]
[20]
[14]
[27]
[28]
[1]
[11]
[14]
[29]
[1]
[10]
[16]
[10]
[18]
[23]
[28]
[29]
[1]
[29]
[17]
[14]
[1]
[25]
[27]
[14]
[31]
[10]
[18]
[21]
[18]
[23]
[16]
[1]
[32]
[18]
[28]
[13]
[24]
[22]
[1]
[10]
[23]
[13]
[1]
[11]
[24]
[30]
[16]
[17]
[29]
[1]
[10]
[28]
[28]
[14]
[29]
[28]
[1]
[32]
[17]
[14]
[23]
[1]
[24]

KeyboardInterrupt: 

In [ ]:
global block_num
block_num = 0

In [ ]:
vocab = pd.read_csv("{0}/{1}".format(data_path, vocab_file),
                    header=None)
vocab_size = len(vocab)